# Numerical modelling of synaptic transmission in 3D

In [1]:
%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt
from getplate import getPlate
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, clear_output

In [2]:
%%html 
<style>
.output_wrapper button.btn.btn-default, .output_wrapper .ui-dialog-titlebar {
    display: none;
} </style>

## 3D
* Build mass matrix and load vector
* Need new mesh generator, new matrix and load vector builder, otherwise the same
* Solve system of ODE's
* More complicated animation